In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import glob

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image
import torchvision
import torch

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet.git  # clone repo
%cd /content/Yolov5_StrongSORT_OSNet
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

#clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'Yolov5_StrongSORT_OSNet'...
remote: Enumerating objects: 1764, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 1764 (delta 72), reused 108 (delta 62), pack-reused 1638
Receiving objects: 100% (1764/1764), 36.08 MiB | 13.99 MiB/s, done.
Resolving deltas: 100% (908/908), done.
Submodule 'strong_sort/deep/reid' (https://github.com/KaiyangZhou/deep-person-reid) registered for path 'strong_sort/deep/reid'
Submodule 'yolov5' (https://github.com/ultralytics/yolov5.git) registered for path 'yolov5'
Cloning into '/content/Yolov5_StrongSORT_OSNet/strong_sort/deep/reid'...
remote: Enumerating objects: 9860, done.        
remote: Counting objects: 100% (10/10), done.        
remote: Compressing objects: 100% (9/9), done.        
remote: Total 9860 (delta 1), reused 9 (delta 1), pack-reused 9850        
Receiving objects: 100% (9860/9860), 9.57 MiB | 13.96 MiB/s, done.
Resolving deltas: 100% (7286/7286), done.
Clonin

# Download data

Get test video from repo and extract the first 2 seconds of it 

In [3]:
# extract 3 seconds worth of video frames of it
!yes | ffmpeg -ss 00:00:00 -i '/content/drive/MyDrive/2022_Selab/MediaEval/code/VISEM_Tracking_Train_v4/Train/60/60.mp4' -t 00:00:20 -c copy outvideo60.avi

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [4]:
!python track.py --yolo-weights '/content/drive/MyDrive/2022_Selab/MediaEval/code/yolov5/runs/train/exp/weights/best.pt' --strong-sort-weights osnet_x0_25_msmt17.pt --source outvideo60.avi --save-vid

strong_sort/deep/reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Downloading...
From: https://drive.google.com/uc?id=1Kkx2zW89jq_NETu4u42CFZTMVD5Hwm6e
To: /content/Yolov5_StrongSORT_OSNet/osnet_x0_25_msmt17.pt
100% 9.34M/9.34M [00:00<00:00, 139MB/s]
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


# Show results

https://stackoverflow.com/questions/60977179/how-to-play-avi-file-in-google-colab

Convert avi to mp4

In [5]:
!ffmpeg -i /content/Yolov5_StrongSORT_OSNet/runs/track/exp/outvideo60.mp4 outputVideo60.mp4

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

Get the file content into data_url

In [6]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('outputVideo60.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

Display it with HTML

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)